---
## Assignment 1.5: Regression and Correlation

Which of your Personal Focus Crimes share the most similar weekly rhythm — and which are completely out of sync?

*Draws from*: Week 4, Exercises 3.1, 3.2, and 3.3.

Select **at least 4** of your Personal Focus Crimes for this exercise. More is fine - if you want to be closer to the actual weekly exercise.

> * For each selected crime type, compute the total number of incidents for each of the **168 hours of the week** (i.e., Monday 00:00–01:00, Monday 01:00–02:00, ..., Sunday 23:00–00:00). Each crime type will give you a vector of 168 values.
>
> * **Scatterplot matrix**: Create a grid of pairwise scatterplots — one panel per pair of crime types, with one crime on each axis and each of the 168 hours as a point. Label each panel clearly with the two crime types being compared.
>
> * **Linear regression**: Using the closed-form equations from Week 4 (not a library), fit a regression line $y = ax + b$ to each pair and add it to the relevant panel:
>   $$a = \frac{\sum_i x_i y_i - N\langle x\rangle\langle y\rangle}{\sum_i x_i^2 - N\langle x\rangle^2}, \qquad b = \langle y \rangle - a\langle x \rangle$$
>
> * **$R^2$**: Compute $R^2$ for each pair and display the value on each panel. 
>
> * Looking at your results: which pair of crimes is **most correlated**? Which is **least correlated**? Does the answer match your intuition? Why might two crime types share a weekly rhythm — or not?

In [ ]:
import pandas as pd
df = pd.read_csv('merged_sfpd.csv')
df['incident_category'] = df['incident_category'].replace('Larceny Theft', 'LARCENY/THEFT')

In [ ]:
df[]

In [ ]:

personal_focus = [
    'LARCENY/THEFT',
    'NON-CRIMINAL',
    'ASSAULT',
    'VEHICLE THEFT',
    'DRUG/NARCOTIC',
    'VANDALISM',
    'WARRANTS',
    'BURGLARY',
    'SUSPICIOUS OCC'
]

df_pf = df[df['incident_category'].isin(personal_focus)].copy()
df_pf['incident_category'].value_counts()

In [ ]:
df_pf.head()

In [ ]:
df_hour = df_pf['incident_time'].str[0:2].astype(int)
df_pf['hour'] = df_hour
df_pf['day_of_week'] = df_pf['incident_day_of_week']

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

df_pf['day_of_week'] = pd.Categorical(df_pf['day_of_week'], categories=day_order, ordered=True)

hourly_counts = (
    df_pf.groupby(['incident_category', 'day_of_week', 'hour'])
    .size()
    .reset_index(name='count')
)

# Create a 168-hour index: day*24 + hour
hourly_counts['hour_of_week'] = hourly_counts['day_of_week'].cat.codes * 24 + hourly_counts['hour']

hourly_counts = (
    hourly_counts.pivot_table(index='hour_of_week', columns='incident_category', values='count', fill_value=0)
)

hourly_counts.columns.name = None
hourly_counts = hourly_counts.sort_index().reset_index(drop=True)

hourly_counts


In [ ]:
from itertools import combinations
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np
import matplotlib.pyplot as plt

# Linear regression from scratch using Equations 1 and 2
def linear_regression(x, y):
    N = len(x)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    a = (np.sum(x * y) - N * x_mean * y_mean) / (np.sum(x**2) - N * x_mean**2)
    b = y_mean - a * x_mean
    return a, b

# Hour-of-week color gradient
hours = np.arange(168)
norm = mcolors.Normalize(vmin=0, vmax=167)
cmap = cm.coolwarm
pairs = list(combinations(personal_focus, 2))

# Create 6x6 subplot grid (A4-ish size)
fig, axes_grid = plt.subplots(6, 6, figsize=(8.27, 11.69))

for idx, (c1, c2) in enumerate(pairs):
    row = idx // 6
    col = idx % 6
    ax = axes_grid[row, col]
    
    x = hourly_counts[c1].values.astype(float)
    y = hourly_counts[c2].values.astype(float)
    
    # Scatter with hour-of-week coloring
    ax.scatter(x, y, s=3, alpha=0.7, c=hours, cmap=cmap, norm=norm)
    
    # Linear regression
    a, b = linear_regression(x, y)
    x_line = np.linspace(x.min(), x.max(), 100)
    y_line = a * x_line + b
    ax.plot(x_line, y_line, color='black', linewidth=0.8, linestyle='--')
    
    # Shorten labels for readability
    label1 = c1[:12]
    label2 = c2[:12]
    ax.set_title(f"{label1}\nvs {label2}", fontsize=4.5, pad=2)
    ax.tick_params(axis='both', labelsize=4)

plt.subplots_adjust(hspace=0.7, wspace=0.4)
plt.suptitle("Pairwise Scatterplots with Linear Fits (colored by hour of week)", 
             fontsize=10, fontweight='bold', y=0.995)

# Add a small colorbar for the hour-of-week gradient
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar_ax = fig.add_axes([0.92, 0.02, 0.015, 0.15])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('Hour of Week', fontsize=6)
cbar.ax.tick_params(labelsize=5)

plt.tight_layout(rect=[0, 0, 0.91, 0.98])
plt.show()

In [ ]:
# R^2 function
def r_squared(x, y):
    a, b = linear_regression(x, y)
    y_pred = a * x + b
    ss_res = np.sum((y - y_pred) ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    if ss_tot == 0:
        return 0.0
    return 1 - ss_res / ss_tot

# Calculate R^2 for all pairs
r2_values = {}
for c1, c2 in pairs:
    x = hourly_counts[c1].values.astype(float)
    y = hourly_counts[c2].values.astype(float)
    r2_values[(c1, c2)] = r_squared(x, y)

# Find most similar and most dissimilar pairs
most_similar = max(r2_values, key=r2_values.get)
most_dissimilar = min(r2_values, key=r2_values.get)

print(f"Most similar pair:    {most_similar[0]} & {most_similar[1]}  (R² = {r2_values[most_similar]:.4f})")
print(f"Most dissimilar pair: {most_dissimilar[0]} & {most_dissimilar[1]}  (R² = {r2_values[most_dissimilar]:.4f})")
print()

# Print all R² values sorted
print("All R² values (sorted):")
for pair, r2 in sorted(r2_values.items(), key=lambda x: x[1], reverse=True):
    print(f"  {pair[0]:25s} vs {pair[1]:25s}  R² = {r2:.4f}")

# --- Mega-plot with R² and highlights ---
fig, axes_grid = plt.subplots(6, 6, figsize=(8.27, 11.69))

for idx, (c1, c2) in enumerate(pairs):
    row = idx // 6
    col = idx % 6
    ax = axes_grid[row, col]
    
    x = hourly_counts[c1].values.astype(float)
    y = hourly_counts[c2].values.astype(float)
    
    # Highlight most similar (green) and most dissimilar (red)
    is_similar = (c1, c2) == most_similar
    is_dissimilar = (c1, c2) == most_dissimilar
    
    if is_similar:
        ax.set_facecolor('#d4edda')
        for spine in ax.spines.values():
            spine.set_edgecolor('green')
            spine.set_linewidth(2)
    elif is_dissimilar:
        ax.set_facecolor('#f8d7da')
        for spine in ax.spines.values():
            spine.set_edgecolor('red')
            spine.set_linewidth(2)
    
    # Scatter with hour-of-week coloring
    ax.scatter(x, y, s=3, alpha=0.7, c=hours, cmap=cmap, norm=norm)
    
    # Linear regression line
    a, b = linear_regression(x, y)
    x_line = np.linspace(x.min(), x.max(), 100)
    y_line = a * x_line + b
    ax.plot(x_line, y_line, color='black', linewidth=0.8, linestyle='--')
    
    # R² text
    r2 = r2_values[(c1, c2)]
    ax.text(0.05, 0.92, f"R²={r2:.3f}", transform=ax.transAxes, fontsize=4.5,
            verticalalignment='top', fontweight='bold',
            color='green' if is_similar else ('red' if is_dissimilar else 'black'),
            bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7, linewidth=0.3))
    
    label1 = c1[:12]
    label2 = c2[:12]
    ax.set_title(f"{label1}\nvs {label2}", fontsize=4.5, pad=2)
    ax.tick_params(axis='both', labelsize=4)

plt.subplots_adjust(hspace=0.7, wspace=0.4)
plt.suptitle("Pairwise Scatterplots with Linear Fits & R²\n(green = most similar, red = most dissimilar)", 
             fontsize=10, fontweight='bold', y=0.998)

# Colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar_ax = fig.add_axes([0.92, 0.02, 0.015, 0.15])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('Hour of Week', fontsize=6)
cbar.ax.tick_params(labelsize=5)

plt.tight_layout(rect=[0, 0, 0.91, 0.98])
plt.show()